# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [1]:
# скачиваем корпус
!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt.zip -d data/

mkdir: cannot create directory ‘data’: File exists
--2024-09-29 17:33:15--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip [following]
--2024-09-29 17:33:15--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4723095 (4.5M) [application/zip]
Saving to: ‘data/corpus_wsd_50k.txt.zip.2’

corpus_wsd_50k.txt. 100%[===================>]   

In [2]:
# вычитываем корпус в память
corpus = open('data/corpus_wsd_50k.txt').read()
print(corpus[:200])

	how	How
long%3:00:02::	long	long
	have	has
	it	it
be%2:42:03::	be	been
	since	since
	you	you
review%2:31:00::	review	reviewed
	the	the
objective%1:09:00::	objective	objectives
	of	of
	you	your
benefi


In [3]:
# парсим корпус так, чтобы в итоге он представлял собой список с предложениями,
# преложения были бы списками со словами,
# а слова были бы списками из трех элементов -- значения (если слово не однозначно), леммы и самого слова, как оно употреблено в предложении
corpus_wsd = []
corpus = corpus.split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
corpus_wsd[1]

[['', 'have', 'Have'],
 ['', 'you', 'you'],
 ['permit%2:41:00::', 'permit', 'permitted'],
 ['', 'it', 'it'],
 ['', 'to', 'to'],
 ['become%2:42:01::', 'become', 'become'],
 ['', 'a', 'a'],
 ['giveaway%1:21:00::', 'giveaway', 'giveaway'],
 ['program%1:09:01::', 'program', 'program'],
 ['rather%4:02:02::', 'rather', 'rather'],
 ['', 'than', 'than'],
 ['', 'one', 'one'],
 ['', 'that', 'that'],
 ['have%2:42:00::', 'have', 'has'],
 ['', 'the', 'the'],
 ['goal%1:09:00::', 'goal', 'goal'],
 ['', 'of', 'of'],
 ['improved%3:00:00::', 'improved', 'improved'],
 ['employee%1:18:00::', 'employee', 'employee'],
 ['morale%1:26:00::', 'morale', 'morale'],
 ['', 'and', 'and'],
 ['', ',', ','],
 ['consequently%4:02:00::', 'consequently', 'consequently'],
 ['', ',', ','],
 ['increased%3:00:00::', 'increased', 'increased'],
 ['productivity%1:07:00::', 'productivity', 'productivity'],
 ['', '?', '?']]

In [1]:
# !pip install nltk -U

In [82]:
# импортируем nltk
import nltk

# импортируем wordnet, чтобы с помощью него искать определения словам
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
# выгружаем список стоп-слов, чтобы убирать их из предложений
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [85]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [150]:
nltk.download('punkt_tab')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

from typing import Union

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def sentence2bow(sent:Union[str,list]) -> list:
  """
  Превращает предложение в виде строки или список слов
  в список лемм (изначально без стоп-слов и пунктуации)
  """
  res = []
  if isinstance(sent, str):
    sent = word_tokenize(sent.lower())
  for token in sent:
    # if token not in stopwords and token not in string.punctuation:
    res.append(token.lower())
  return res

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Жаккар

In [147]:
# функция для подбора наиболее подходящего значения на основе количества слов,
# использованных в определении и предложении.
# Если общих слов нет, то возвращается Null

def lesk_jaccard(word, context:list):
  max_jaccard_score = 0
  best_jaccard_key = None

  for synset in wn.synsets(word):
    word_key = synset.lemmas()[0].key()
    word_definition = sentence2bow(synset.definition())

    intersection = (set(word_definition) & set(context))
    union = (set(word_definition) | set(context))
    jaccard_score =  len(intersection) / len(union)

    if jaccard_score > max_jaccard_score:
      max_jaccard_score = jaccard_score
      best_jaccard_key = word_key

    return best_jaccard_key

In [151]:
# вычисляем точность алгоритма lesk_jaccard

total = 0  # общее количество неоднозначных слов
correct = 0  # количество слов, для которых правильно подобраны значения

for doc in corpus_wsd:
  if doc != [['']]:
    sent = sentence2bow([t[1] for t in doc])
    for w in doc:
      if w[0] != '': # если слово не однозначно
        correct += (lesk_jaccard(w[1].lower(), sent) == w[0]) * 1  # если подобранное алгоритмом значение совпадает со значением, указанным вкорпусе, то к correct прибавляется 1
        total += 1  # увеличиваем количество обработанных неоднозначных слов

accuracy = correct / total

print('Слов рассмотрено: ' + '{:,}'.format(total),
      'Из них правильно определено: ' + '{:,}'.format(correct),
      'Accuracy: ' + "{:.2%}".format(accuracy),
      sep = '\n'
)

Слов рассмотрено: 239,913
Из них правильно определено: 61,914
Accuracy: 25.81%


Эксперименты с предобработкой предложений из корпуса и определений дали следующие результаты:

- токенизация определений с помощью word_tokenize(), с очисткой от стоп-слов, приведением к нижнему регистру и очисткой от пунктуации -- 2.6%
- без какой-либо обработки -- только split(), предложения из корпуса составляются из [1] элементов -- 25.24%
- без какой-либо обработки -- только split(), предложения из корпуса составляются из [2] элементов -- 23.61%
- токенизация определений с помощью word_tokenize(), очистка от пунктуации и приведение к нижнему регистру, предложения из корпуса составляются из [2] элементов -- 24.26%
- ЛУЧШИЙ РЕЗУЛЬТАТ: только токенизация определений с помощью word_tokenize() и приведение к нижнему регистру, предложения из корпуса составляются из [1] элементов -- 25.81%

### Косинусное расстояние

In [114]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [115]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [116]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [162]:
import numpy

definition1 = 'a system of projects or services intended to meet a public need'
definition2 = '(computer science) a sequence of instructions that a computer can interpret and execute'
context = 'i wrote a computer _ that predicts the sense of a word'

definition1_emb = embed(definition1)
definition2_emb = embed(definition2)
context_emb = embed(context)

sims = cosine_similarity(context_emb.reshape(1, -1), [definition1_emb, definition2_emb])
sims

array([[0.10057068, 0.3182024 ]], dtype=float32)

In [164]:
max(sims[0])

0.3182024

In [200]:
# функция для подбора наиболее подходящего значения на основе близости векторов текстов

def lesk_cosine(word, context:str):
    max_similarity_score = 0
    best_key = None

    context_emb = embed(context)

    word_keys = []
    definition_embedings = []

    for synset in wn.synsets(word):
      word_keys.append(synset.lemmas()[0].key())

      definition_embedings.append(embed(synset.definition()))

    similarity_score = cosine_similarity(context_emb.reshape(1, -1), definition_embedings)[0]

    pairs = list(zip(word_keys, similarity_score))

    for p in pairs:
      if p[1] > max_similarity_score:
        max_similarity_score = p[1]
        best_key = p[0]

    return best_key

In [204]:
corpus_wsd_test = [
    [['', 'be', 'Is'], ['', 'you', 'your'], ['purchasing_agent%1:18:00::', 'purchasing_agent', 'purchasing agent'], ['offer%2:40:02::', 'offer', 'offering'], ['too_much%4:02:00::', 'too_much', 'too much'], ['free%3:00:00::', 'free', 'free'], ['buying%1:04:00::', 'buying', 'buying'], ['service%1:14:05::', 'service', 'service'], ['', 'for', 'for'], ['employee%1:18:00::', 'employee', 'employees'], ['', '?', '?']]
]

In [206]:
# вычисляем точность алгоритма lesk_cosine

total = 0  # общее количество неоднозначных слов
correct = 0  # количество слов, для которых правильно подобраны значения

for doc in corpus_wsd_test:
  if doc != [['']]:
    sent = ' '.join([t[2] for t in doc])
    for w in doc:
      if w[0] != '': # если слово не однозначно
        try:
          correct += (lesk_cosine(w[2].lower(), sent) == w[0]) * 1  # если подобранное алгоритмом значение совпадает со значением, указанным вкорпусе, то к correct прибавляется 1
          total += 1  # увеличиваем количество обработанных неоднозначных слов
        except Exception as e:
          print(e)
          print(w)
          print(doc, '\n')

accuracy = correct / total

print('Слов рассмотрено: ' + '{:,}'.format(total),
      'Из них правильно определено: ' + '{:,}'.format(correct),
      'Accuracy: ' + "{:.2%}".format(accuracy),
      sep = '\n'
)

Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
['purchasing_agent%1:18:00::', 'purchasing_agent', 'purchasing agent']
[['', 'be', 'Is'], ['', 'you', 'your'], ['purchasing_agent%1:18:00::', 'purchasing_agent', 'purchasing agent'], ['offer%2:40:02::', 'offer', 'offering'], ['too_much%4:02:00::', 'too_much', 'too much'], ['free%3:00:00::', 'free', 'free'], ['buying%1:04:00::', 'buying', 'buying'], ['service%1:14:05::', 'service', 'service'], ['', 'for', 'for'], ['employee%1:18:00::', 'employee', 'employees'], ['', '?', '?']] 

Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
['too_much%4:02:00::', 'too_much', 'too much']
[['', 'be', 'Is'], ['', 'you', 'your'], ['purchasing_agent%1:18:00::', 'purchasing_a

In [212]:
lesk_cosine('too much'.lower(), ' '.join([t[2] for t in corpus_wsd_test]))

TypeError: sequence item 0: expected str instance, list found

In [ ]:
for w in

# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты.